In [1]:
import os
import time
from glob import glob
import pandas as pd

In [2]:
path = '../../graphs_site'
date = '220923'

In [3]:
date_formatted = f'{date[-2:]}/{date[2:4]}/{date[:2]}'

### Create index.html

In [4]:
sections = sorted(pd.read_csv(f'{date}_all_routes.csv')["section"].unique().tolist())
sections_formatted = [s.lower().replace(" ","").replace("'","") for s in sections]

In [5]:
# ## Needed to run only once - to create the template itself with the sections dropdown

# with open(os.path.join(path,f'index_template.html'),'r') as f:
#     html_index = f.read()
    
# filler = ''
# for i,s in enumerate(sections):
#     filler+= f'<li><a class="dropdown-item" href="{sections_formatted[i]}.html">{s}</a></li>'
#     #filler+= f'<a class="dropdown-item" href="{sections_formatted[i]}.html">{s}</a>'

# html_index = html_index.replace('filler_sections',filler)

# with open(os.path.join(path,f'index_template.html'), 'w') as f:
#     f.write(html_index)   

In [29]:
with open(os.path.join(path,'index_template.html'),'r') as f:
    html_index = f.read()

In [30]:
html_index = html_index.replace('filler_date', date_formatted)

with open(f'{date}_nRoutes_per_grade.html', 'r') as f:
    html_index = html_index.replace('filler_nRoutes_per_grade', f.read())
    
with open(f'{date}_mean_nsends_per_grade.html', 'r') as f:
    html_index = html_index.replace('filler_mean_nSends_per_grade', f.read())
    
with open(f'{date}_total_nsends_per_grade.html', 'r') as f:
    html_index = html_index.replace('filler_total_nSends_per_grade', f.read())
    
with open(f'{date}_holdType_per_grade.html', 'r') as f:
    html_index = html_index.replace('filler_holdType_per_grade', f.read())

with open(f'{date}_holdType_per_grade_normalized.html', 'r') as f:
    filler = f.read()
    filler += '<p>* Can add up to more than 100% as routes can have more than one type of hold.</p>'
html_index = html_index.replace('filler_normalized_holdType_per_grade', filler)
    
with open(f'{date}_style_per_grade.html', 'r') as f:
    html_index = html_index.replace('filler_style_per_grade', f.read())

with open(f'{date}_style_per_grade_normalized.html', 'r') as f:
    filler = f.read()
    filler += '<p>* Can add up to more than 100% as routes can have more than one style.</p>'
html_index = html_index.replace('filler_normalized_style_per_grade', filler)

time.sleep(1)

In [31]:
with open(os.path.join(path,'index.html'), 'w') as f:
    f.write(html_index)

### Create grade summary page

In [52]:
levels = ['6A','6A+','6B','6B+','6C','6C+','7A','7A+']

In [53]:
with open(os.path.join(path,'grade_summary_template.html'),'r') as f:
    html_grades = f.read()

In [54]:
html_grades = html_grades.replace('filler_date', date_formatted)

## table:
with open(f'{date}_height_summary_table.html', 'r') as f:
    html_grades = html_grades.replace('filler_table_grade_height', f.read())

## histograms:
fill_str = ''
with open(f'{date}_sendLevel_hist_around_route_grade_allGrades.html', 'r') as ff:
    fill_str+= f'{ff.read()}'
    
with open(f'{date}_sendLevel_hist_around_route_grade_allGrades_curves.html', 'r') as ff:
    fill_str+= f'{ff.read()}'

for g in levels:
    with open(f'{date}_sendLevel_hist_around_route_grade_{g}.html', 'r') as ff:
        fill_str+= f'{ff.read()}'

html_grades = html_grades.replace('filler_histograms', fill_str)

## heatmaps
for g in levels:
    with open(f'{date}_heatmap_xLevel_yHeight_hovAttempts_{g}.html', 'r') as f:
        html_grades = html_grades.replace(f'filler_{g}_summary', f.read())
    

In [55]:
with open(os.path.join(path,'grade_summary.html'), 'w') as f:
    f.write(html_grades)

### Create section pages

In [62]:
plot_types = ['_heatmap_xLevel_yHeight_hovAttempts_',
              '_xAttempts_yHeight_hueLevel_',
             '_xLevels_yHeight_hueAtt_']

In [63]:
all_files = [[sorted(glob(f'{date}{p_t}{s}_*.html')) for p_t in plot_types] for s in sections_formatted]

In [64]:
for i,s in enumerate(sections_formatted):
    with open(os.path.join(path,f'section_template.html'),'r') as f:
        html_section = f.read()
        
    html_section = html_section.replace('filler_date', date_formatted)
    html_section = html_section.replace('filler_section_name', sections[i])
    
    fill_str = ''
    
    for j in range(len(all_files[i][0])):
        fill_str += '<div class="row">'
        for k,p_t in enumerate(plot_types):
            with open(f'{date}{p_t}{s}_{j:02d}.html', 'r') as ff:
                fill_str+= f'<div class="col-sm">{ff.read()}</div>'
        fill_str+= '</div>'
    
    html_section = html_section.replace('filler_routes_graphs', fill_str)
    
    with open(os.path.join(path,f'{s}.html'), 'w') as f:
        f.write(html_section)
    

### Create individual routes normed heat map page

In [100]:
levels = ['6A','6A+','6B','6B+','6C','6C+','7A','7A+']

all_files = [sorted(glob(f'{date}_heatmap_normed_xLevel_yHeight_hovAttempts_{l}_*.html')) for l in levels]


In [102]:
with open(os.path.join(path,f'individual_routes_normed_template.html'),'r') as f:
    html_indiv_routes_normed = f.read()
    
html_indiv_routes_normed = html_indiv_routes_normed.replace('filler_date', date_formatted)

fill_str = ''

for j in range(len(all_files)):

    ## add the grade summary heat map
    fill_str += '<div class="row">'
    fill_str += f'<h4>All heat maps {levels[j]}</h4>'
    with open(f'{date}_heatmapT_xLevel_yHeight_hovAttempts_{levels[j]}.html', 'r') as f:
        temp = f.read()

    fill_str+= temp.replace('style="height', 'style="margin:auto;height')
    fill_str+= '</div>'

    fill_str += '<div class="row">'
    ii=0
    while ii<len(all_files[j]):
        with open(all_files[j][ii], 'r') as ff:
            fill_str+= f'<div class="col-sm">{ff.read()}</div>'
        ii+=1
    fill_str+= '</div>'
    fill_str+= '<br/><br/>'

html_indiv_routes_normed = html_indiv_routes_normed.replace('filler_routes_graphs', fill_str)

with open(os.path.join(path,f'individual_routes_normed.html'), 'w') as f:
    f.write(html_indiv_routes_normed)